In [ ]:
%matplotlib inline
import sys
import numpy as np
import numpy.random as rnd
import time
import GPflow
import datasets
import tensorflow as tf

In [ ]:
M = 50

In [ ]:
def func(x):
    return np.sin(x * 3*3.14) + 0.3*np.cos(x * 9*3.14) + 0.5 * np.sin(x * 7*3.14)
X = rnd.rand(10000, 1) * 2 - 1
Y = func(X) + rnd.randn(10000, 1) * 0.2
plt.plot(X, Y, 'x')
D = X.shape[1]
Xt = np.linspace(-1.1, 1.1, 100)[:, None]
Yt = func(Xt)

In [ ]:
def init():
    kern = GPflow.kernels.RBF(D, 1)
    Z = X[:M, :]
    m = GPflow.svgp.SVGP(X, Y, kern, GPflow.likelihoods.Gaussian(), Z)
    return m
m = init()

In [ ]:
m.minibatch = len(X)
m._compile()
ground_truth = m._objective(m.get_free_state())[0]
print ground_truth

In [ ]:
mbps = np.logspace(-2, 0, 10)
times = []
for mbp in mbps:
    m.minibatch = int(len(X) * mbp)
    m._compile()
    start_time = time.time()
    objs = [m._objective(m.get_free_state())[0] for _ in xrange(20)]
#    plt.hist(objs, bins = 100)
#    plt.axvline(ground_truth, color='r')
    times.append(time.time() - start_time)

In [ ]:
plt.plot(mbps, times, 'x-')
plt.xlabel("Minibatch proportion")
plt.ylabel("Time taken")

In [ ]:
class DAdamOptimizer(tf.train.AdamOptimizer):
      def _valid_dtypes(self):
            return set([tf.float32, tf.float64])

class DGradientDescentOptimizer(tf.train.GradientDescentOptimizer):
      def _valid_dtypes(self):
            return set([tf.float32, tf.float64])

In [ ]:
opt_list = [{"method": "L-BFGS-B", "minibatch": len(X)},
            {"method": tf.train.AdamOptimizer(), "cast": True, "minibatch": 1000},
            {"method": tf.train.GradientDescentOptimizer(), "cast": True, "minibatch": 1000}]
for opt in opt_list[1:2]:
    m = init()
    m._compile()
    print m._objective(m.get_free_state())[0]
    if "minibatch" in opt:
        m.minibatch = opt['minibatch']
    m._needs_recompile = True
    ilist = []
    tlist = []
    flist = []
    start_time = time.time()
    def cb(x):
        if cb.iteration % 10 == 0:
            f = m._objective(x)[0]
            flist.append(f)
            ilist.append(cb.iteration)
            tlist.append(time.time() - start_time)
            sys.stdout.write('\r' + str((cb.iteration, f)))
            sys.stdout.flush()
        cb.iteration += 1
    cb.iteration = 0
    m.optimize(max_iters=int(1000 * float(len(X)) / m.minibatch), callback=cb, **opt)
    plt.figure(1)
    plt.subplot(121)
    plt.plot(ilist, flist)
    plt.subplot(122)
    plt.plot(tlist, flist)

# m.optimize(method=DGradientDescentOptimizer(tf.constant(0.0001, tf.float64)), max_iters=10, callback=cb)
# m.optimize(method=tf.train.GradientDescentOptimizer(0.0001), max_iters=10, callback=cb, cast=True)
# m.optimize(method=tf.train.AdamOptimizer(), max_iters=10, callback=cb, cast=True)
# m.optimize(method=DAdamOptimizer(), max_iters=10, callback=cb, cast=False)

In [ ]:
print m._objective(m.get_free_state())[0]
x = m.get_free_state()
x[:5] = 0.0
print m._objective(x)[0]

In [ ]:
x

In [ ]:
ilist